In [33]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 


In [35]:
import numpy as np

In [36]:
temp = X['Date']+' '+X['Time']
X['temp']=temp

In [37]:
total_atipicos=[]
for column in X:
    if column == "Time" or column == "Date" or column == "temp":
        continue

    x_temp = X[column]
    q3=np.quantile(x_temp, 0.75)
    q1=np.quantile(x_temp, 0.25)
    limsp=q3+1.5*(q3-q1)
    liminf=q1-1.5*(q3-q1)
    for i in range(0, len(x_temp)):
        if x_temp[i]>limsp:
            if i not in total_atipicos:
                total_atipicos.append(i)
        if x_temp[i]<liminf:
            if i not in total_atipicos:
                total_atipicos.append(i)    

In [38]:
len(total_atipicos)

3895

In [39]:
X_new = X.drop(total_atipicos)
X_new = X_new.reset_index()

In [40]:
X_new

,index,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,temp
0,184,3/18/2004,10:00:00,4.5,1617,-200,21.3,1333,349,686,150,2010,1819,17.8,40.5,0.8210,3/18/2004 10:00:00
1,185,3/18/2004,11:00:00,2.8,1473,-200,14.3,1127,224,831,152,1752,1568,20.8,34.4,0.8365,3/18/2004 11:00:00
2,186,3/18/2004,12:00:00,2.2,1379,-200,12.5,1068,171,899,139,1663,1374,23.8,28.2,0.8219,3/18/2004 12:00:00
3,187,3/18/2004,13:00:00,2.2,1385,-200,12.2,1056,149,891,133,1648,1268,24.2,28.7,0.8515,3/18/2004 13:00:00
4,188,3/18/2004,14:00:00,2.3,1379,-200,13.1,1087,137,901,126,1660,1144,25.2,24.9,0.7829,3/18/2004 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5457,9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568,4/4/2005 10:00:00
5458,9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119,4/4/2005 11:00:00
5459,9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406,4/4/2005 12:00:00
5460,9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139,4/4/2005 13:00:00


In [41]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()

In [43]:
X_new=X_new.drop("index", axis="columns")

In [44]:
import itertools

def regression_model_combs (column): 
    excluded_cols = ["temp", "Date", "Time", column]
    included_cols = [col for col in X_new.columns if col not in excluded_cols]

    total_R2 = []

    for combination in range(1,6):
        for combo in itertools.combinations(included_cols, combination):
            X1 = X_new.copy()
            input=X1[list(combo)]
            regr.fit(input, X1[column])
            Y_predict=(regr.predict(input))
            X1['predict']=Y_predict
            r2=r2_score(X1[column], X1['predict'])
            total_R2.append({
                "name": f'{combo}',
                "r2_score": r2
            })
    sorted_r2 = sorted(total_R2, key=lambda x: x['r2_score'])
    print("Para la columna ", column, "el mejor modelo es: ", sorted_r2[-1])

In [45]:
regression_model_combs("CO(GT)")

Para la columna  CO(GT) el mejor modelo es:  {'name': "('PT08.S1(CO)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T')", 'r2_score': 0.8677095037782487}


PUNTO 1
El mejor modelo de regresión es el realizado con las columnas PT08.S1(CO), C6H6(GT), NOx(GT), NO2(GT) y T, obteniendo un R2 de 0.8677095037782487.

In [46]:
regression_model_combs("C6H6(GT)")

Para la columna  C6H6(GT) el mejor modelo es:  {'name': "('CO(GT)', 'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'NO2(GT)')", 'r2_score': 0.9821296010527406}


PUNTO 2
El mejor modelo de regresión, con un R2 > 0.7 es el realizado con las columnas CO(GT), PT08.S1(CO), PT08.S2(NMHC), PT08.S3(NOx) y NO2(GT), obteniendo un R2 de 0.9821296010527406.